# Car Tracker 


This notebook shows how to build a simple car dwell time tracker a pretrained SSD Mobilenet Tensorflow Object Detection Model and a Centroid Tracker.

By completing this notebook, you will learn:
* How to write a Python script for your app that takes in camera streams, performs inference, and outputs results
* How to use a Tensorflow Object Detecrtion model with Panorama
* How to bundle additional Python files and libraries with your container
* How to build a simple car tracker
* How to test your code using the Panorama emulator, which saves you build and deploy time
* How to programmatically package and deploy applications using the Panorama CLI

--- 

1. [Prerequisites](#Prerequisites)
1. [Set up](#Set-up)
1. [Import model](#Import-model)
1. [Write and test app code](#Write-and-test-app-code)
1. [Package app](#Package-app)
1. [Deploy app to device](#Deploy-app-to-device)

# Prerequisites

1. In a terminal session on this Jupyter notebook server, run `aws configure`. This allows this notebook server to access Panorama resources on your behalf.

In [ ]:
# Run this only once
!pip3 install scipy

# Set Up
Import libraries for use with this notebook environment, you do not need these libraries when you write your application code. Run these 3 cells every time you update your app code and restart your kernel.

In [ ]:
import boto3
from botocore.exceptions import ClientError
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')
panorama_client = boto3.client('panorama')

import os
import time
import json
import sagemaker
import sys

path = os.path.abspath(os.path.join(os.path.dirname("panorama_sdk"), '../'))
sys.path.insert(1, path + '/panorama_sdk')
import utils

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,20)
from IPython.display import clear_output
from IPython.core.magic import register_cell_magic

@register_cell_magic
def run_and_save(line, cell):
    'Run and save python code block to a file'
    with open(line, 'wt') as fd:
        fd.write(cell)
    code = compile(cell, line, 'exec')
    exec(code, globals())

## Notebook parameters

Global constants that help the notebook create Panorama resources on your behalf.

In [ ]:
## Device Related Globals
DEVICE_ID = '' # Device ID, should look like: device-oc66nax4cgzwhyuaeyifrqowue
assert DEVICE_ID != '', "PLEASE ENTER  YOUR DEVICE ID"

# Bucket Related Globals
S3_BUCKET = '' # Enter your S3 bucket info here
assert S3_BUCKET != '', "PLEASE ENTER A BUCKET NAME"

# AWS region
AWS_REGION = 'us-east-1' # Enter your desired AWS region


ML_MODEL_FNAME = 'ssd_mobilenet_v2_coco'

In [ ]:
## Working directory
__builtins__.workdir = '/home/ubuntu/awspanoramasamples/samples'

## APP Graph Related Globals

__builtins__.project_name = 'car_tracker'
__builtins__.package_name = 'CAR_TRACKER_CODE'
__builtins__.camera_node_name = 'RTSP_STREAM'
__builtins__.model_node_name = 'CAR_SSD_TF_MODEL'

## Model Related Globals
__builtins__.model_preprocess_size = 300
__builtins__.emulator_model_name = ML_MODEL_FNAME + '-LINUX_ARM64'

## Video Related Globals
__builtins__.videoname = 'Car-in-the-mist_coverr.mp4'

### Account ID
__builtins__.account_id = boto3.client("sts").get_caller_identity()["Account"]

### DEPLOYED APPLICATION NAME 
__builtins__.app_name = 'car_tracker_app'

## Set up application

Every application uses the creator's AWS Account ID as the prefix to uniquely identifies the application resources. Running `panorama-cli import-application` replaces the generic account Id with your account Id.

In [ ]:
utils.cleanupipynb(os.curdir)
os.chdir('{}/{}/{}'.format(workdir,project_name,app_name))

In [ ]:
!panorama-cli import-application

In [ ]:
os.chdir('{}/{}'.format(workdir,project_name))

# Import model

We need to compile and import the model twice. Once for testing with this notebook server and once for deploying to the Panorama device.

While working with the Panorama sample code, we provide pretrained models for you to use. Locally, models are stored in `panorama_sdk/models`. This step downloads the model artifacts from our Amazon S3 bucket to the local folder. If you want to use your own models, put your tar.gz file into the `panorama_sdk/models folder`.

### Prepare model for testing with notebook server

In [ ]:
# Downloads pretrained model to this notebook
utils.download_model(ML_MODEL_FNAME)

Uploads the model to S3 in order to start the compilation job.

In [ ]:
s3.meta.client.upload_file('{}/panorama_sdk/models/{}.tar.gz'.format(workdir,ML_MODEL_FNAME), 
                           S3_BUCKET, '{}/{}.tar.gz'.format(project_name, ML_MODEL_FNAME))

Start the compilation job. For the Panorama samples, we provide a utils.py library with common functions. We encourage you to look at the code inside utils.py in order to learn what the common functions are doing `panorama_sdk/utils.py`.

In [ ]:
utils.compile_model(region = AWS_REGION, 
                    s3_model_location = 's3://{}/{}/{}.tar.gz'.format(S3_BUCKET, project_name, ML_MODEL_FNAME), 
                    data_shape =  '{"image_tensor":[1,300,300,3]}', 
                    framework = 'TENSORFLOW', 
                    s3_output_location = 's3://{}/{}/'.format(S3_BUCKET, project_name), 
                    role = utils.resolve_sm_role())

Downloads the compiled model for use with this notebook server.

In [ ]:
s3_client.download_file(S3_BUCKET, 
                '{}/{}-LINUX_ARM64.tar.gz'.format(project_name, ML_MODEL_FNAME), 
                '{}/panorama_sdk/models/{}-LINUX_ARM64.tar.gz'.format(workdir,ML_MODEL_FNAME))

Unzip the compiled model

In [ ]:
path = os.getcwd()
os.chdir('{}/panorama_sdk/models'.format(workdir))
!tar -xf {ML_MODEL_FNAME}-LINUX_ARM64.tar.gz --one-top-level
os.chdir(path)

### Prepare model for deploying to Panorama device


In [ ]:
os.chdir('{}/{}/{}'.format(workdir,project_name,app_name))
utils.cleanupipynb(os.curdir)

In [ ]:
asset_name = 'model_asset'
package_path = 'packages/{}-{}-1.0'.format(account_id, model_node_name)
descriptor_path = 'packages/{}-{}-1.0/descriptor.json'.format(account_id, model_node_name)

In [ ]:
!sudo panorama-cli add-raw-model --model-asset-name {asset_name} \
                                --model-s3-uri s3://{S3_BUCKET}/{project_name}/{ML_MODEL_FNAME}.tar.gz \
                                --descriptor-path {descriptor_path}  \
                                --packages-path {package_path}

In [ ]:
os.chdir('{}/{}'.format(workdir,project_name))

# Write and test app code

Every app has an entry point script, written in Python that pulls the frames from camera streams, performs inference, and send the results to the desired location. This file can be found in `your_app/packages/code_node/src/main.py`. Below, you will iterate on the code from within the notebook environment. The entry point file will be updated everytime you run the next notebook cell thanks to the `%%run_and_save people_counter_main.py`. This is a utility function to update the contents of the entry point script. 

The next cell will loop through the test video until you select Stop on your Jupyter notebook cell. 

### Iterating on Code Changes

To iterate on the code:
1. Make changes in the next cell. 
2. Stop the Notebook and Reset the Kernel.
3. Run the 3 Setup cells again before re-running the code cell below.

**CHANGE VIDEO** : For you to change video, please place the video in samples/panorama_sdk/videos and update the global variables with the video and extension name (video.avi for example)

Run only the Set up and Notebook Parameters cells before running this for reiteration

In [ ]:
%%run_and_save app.py

import json
import logging
import time
from logging.handlers import RotatingFileHandler

import boto3
from botocore.exceptions import ClientError
import cv2
import numpy as np
import panoramasdk
import datetime
from CentroidTracker import CentroidTracker

class Application(panoramasdk.node):
    def __init__(self):
        """Initializes the application's attributes with parameters from the interface, and default values."""
        self.MODEL_NODE = "model_node"
        self.MODEL_DIM = 300
        self.frame_num = 0
        self.tracker = CentroidTracker(maxDisappeared=80, maxDistance=90)
        self.tracked_objects = []
        self.tracked_objects_start_time = dict()
        self.tracked_objects_duration = dict()

        try:
            # Parameters
            logger.info('Configuring parameters.')
            self.threshold = self.inputs.threshold.get()
            
            # Desired class
            self.classids = [3.]

        except:
            logger.exception('Error during initialization.')
        finally:
            logger.info('Initialiation complete.')

    def process_streams(self):
        """Processes one frame of video from one or more video streams."""
        self.frame_num += 1
        logger.debug(self.frame_num)

        # Loop through attached video streams
        streams = self.inputs.video_in.get()
        for stream in streams:
            self.process_media(stream)

        self.outputs.video_out.put(streams)

    def process_media(self, stream):
        """Runs inference on a frame of video."""
        image_data = preprocess(stream.image, self.MODEL_DIM)
        logger.debug(image_data.shape)

        # Run inference
        inference_results = self.call({"image_tensor":image_data}, self.MODEL_NODE)

        # Process results (object deteciton)
        self.process_results(inference_results, stream)

    def process_results(self, inference_results, stream):
        """Processes output tensors from a computer vision model and annotates a video frame."""
        if inference_results is None:
            logger.warning("Inference results are None.")
            return
        
        w,h,c = stream.image.shape

        conf_scores = None
        classes = None
        bboxes = None
        rects = []

        for det in inference_results:
            if det.shape[-1] == 4:
                bboxes = det[0]
            elif det.shape[-1] == 100:
                if det[0][0] >= 1:
                    classes = det[0]
                else:
                    conf_scores = det[0]
        
        for a in range(len(conf_scores)):
            if conf_scores[a] * 100 > self.threshold and classes[a] in self.classids:
                (top, left, bottom, right) = bboxes[a]
                rects.append([left*w, top*h, right*w, bottom*h])
                stream.add_rect(left, top, right, bottom)
                
        rects = np.array(rects)
        rects = rects.astype(int)
        objects = self.tracker.update(rects)
        
        logger.info('Tracking {} cars'.format(len(objects)))
        
        for (objectID, bbox) in objects.items():
            x1, y1, x2, y2 = bbox
            x1 = int(x1)
            y1 = int(y1)
            x2 = int(x2)
            y2 = int(y2)

            if objectID not in self.tracked_objects:
                self.tracked_objects.append(objectID)
                self.tracked_objects_start_time[objectID] = datetime.datetime.now()
                self.tracked_objects_duration[objectID] = 0
            else:
                time_diff = datetime.datetime.now() - self.tracked_objects_start_time[objectID]
                sec = time_diff.total_seconds()
                self.tracked_objects_duration[objectID] = sec
            
            duration = self.tracked_objects_duration[objectID]
            
            logger.info('CarId: {} at ({},{}) for {}'.format(objectID, x1, y1, duration))
            stream.add_rect(x1/w, y1/h, x2/w, y2/h)
            stream.add_label('{}s'.format(str(duration)), x1/w, y1/h)

def preprocess(img, size):
    """Resizes and normalizes a frame of video."""
    resized = cv2.resize(img, (size, size))
    x1 = np.asarray(resized)
    x1 = np.expand_dims(x1, 0)
    return x1

def get_logger(name=__name__,level=logging.INFO):
    logger = logging.getLogger(name)
    logger.setLevel(level)
    handler = RotatingFileHandler("/opt/aws/panorama/logs/app.log", maxBytes=100000000, backupCount=2)
    formatter = logging.Formatter(fmt='%(asctime)s %(levelname)-8s %(message)s',
                                    datefmt='%Y-%m-%d %H:%M:%S')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    return logger

def main():
    try:
        logger.info("INITIALIZING APPLICATION")
        app = Application()
        logger.info("PROCESSING STREAMS")
        while True:
            app.process_streams()
    except:
        logger.exception('Exception during processing loop.')

logger = get_logger(level=logging.INFO)
main()

# Package app

Updates the app to be deployed with the recent code

In [ ]:
file_name = 'app.py'
package_path = "{}/{}/{}/packages/{}-{}-1.0/src".format(workdir, project_name, app_name, account_id, package_name)
utils.update_descriptor(project_name, account_id, package_name, file_name)

os.system("cp {} {}/{}".format(file_name, package_path, file_name))
os.system("cp {} {}/{}".format('CentroidTracker.py', package_path, 'CentroidTracker.py'))

### Build app with container

In [ ]:
utils.cleanupipynb(os.curdir)
os.chdir('{}/{}/{}'.format(workdir,project_name,app_name))
container_asset_name = 'code_asset'

In [ ]:
!sudo panorama-cli build --container-asset-name {container_asset_name} \
    --package-path packages/{account_id}-{package_name}-1.0

### Upload application to Panorama for deploying to devices

In [ ]:
utils.cleanupipynb(os.curdir)

### Update camera streams

In the AWS Panorama console, you can select the camera streams, but programmatically, you need to define the camera stream info for the cameras you are using with the app.

Open the ```package.json``` in ```packages/<account_number>-RTSP_STREAM-1.0``` and update the camera username, password and URL. After you have updated your camera credentials, run package-application. You can override this camera stream when you deploy the app.

In [ ]:
!panorama-cli package-application

In [ ]:
os.chdir('{}/{}'.format(workdir,project_name))

### Ready for deploying to a device

Congrats! Your app is now ready to deploy to a device. Next, you can continue in this notebook to deploy the app programmatically or you can go to the Panorama console and deploying using a GUI. The GUI makes it easier to select camera streams and select the devices you want to deploy to. Programmatic deployment is faster to complete and easier to automate.

# Deploy app to device

Let's make sure the device we are deploying to is available.

In [ ]:
response = panorama_client.describe_device(
    DeviceId= DEVICE_ID
)

print('You are deploying to Device: {}'.format(response['Name']))

#### Deploy Application

In [ ]:
utils.deploy_app(DEVICE_ID, project_name, app_name, role=None)

# Clean up

In [ ]:
utils.remove_application(DEVICE_ID)